<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=0ccd2cfdad91291f7c3f8f956c3c8dac105c9cf8c3d74efdcba6dcedd1d91e71
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-05 11:46:33
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16


-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.44 C
-------------------
Today PnL: -92.87 K (-0.64%)
Current PnL: -20.13 L (-13.19%)
CY Booked + Current PnL: -7.48 L (-4.9%)
-------------------
Total profit:  2.10 L
Total loss:  -22.23 L
-------------------
Total Booked + Current PnL: 19.50 L (15.52%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.76%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 87.12 L (60.32%)
Deployed:  1.26 C
Current:  1.44 C
CAGR/XIRR %: 8.21%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-8.41,62.0,X-LC,5.56,229492.0,19126.0,9662.0,-0.92,9.09,4.21,13.68,37.0,1.98,1.60,27.12,XY25,NTT,REFINERIES
79,UNIONBANK,163.00,-5.43,67.0,M-LC,10.18,172256.0,31416.0,13074.0,0.41,22.31,7.59,31.59,66.0,2.40,1.20,54.70,XY24,NTT,BANKS
78,TTKPRESTIG,770.00,103.63,54.0,M-SC,1.72,87770.0,-13007.0,13104.0,-1.28,-12.91,14.93,0.09,245.0,-0.99,0.61,15.31,OX40N,NTT,DURABLES
33,ICICIGI,2252.93,-12.22,64.0,X-MC,5.26,150960.0,14827.0,15760.0,1.61,10.89,10.44,22.47,91.0,0.94,1.05,25.28,X40,ATH,INSURANCE
2,ABBOTINDIA,35195.00,-10.86,39.0,X-MC,2.51,88095.0,-2583.0,17487.0,1.14,-2.85,19.85,16.44,101.0,-0.15,0.61,16.75,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,DABUR,735.00,-7.44,57.0,X-MC,3.05,250502.0,2434.0,105962.0,2.62,0.98,42.30,43.70,102.0,0.02,1.75,15.57,XY24,BTT,FMCG
59,REPCOHOME,880.00,-51.56,76.0,H-SC,6.76,276679.0,-9826.0,267161.0,2.18,-3.43,96.56,89.82,134.0,-0.04,1.93,46.38,XY24,NTT,FINANCE
74,TATAMOTORS,1065.00,-51.85,23.0,X-LC,26.39,151371.0,-125152.0,235231.0,1.71,-45.26,155.40,39.81,54.0,-0.53,1.06,6.69,XY24,NTT,AUTO
33,ICICIGI,2252.93,-12.22,64.0,X-MC,5.26,150960.0,14827.0,15760.0,1.61,10.89,10.44,22.47,91.0,0.94,1.05,25.28,X40,ATH,INSURANCE
34,ICICIPRULI,790.00,-20.19,57.0,X-MC,1.39,137408.0,1620.0,41126.0,1.46,1.19,29.93,31.48,107.0,0.04,0.96,13.46,X40,ATH,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ALKYLAMINE,4546.37,-15.60,37.0,H-SC,4.76,78621.0,-22342.0,116870.0,-3.29,-22.13,148.65,93.63,148.0,-0.19,0.55,17.10,SR,ATH,CHEMICALS
76,TITAGARH,1548.00,-6.04,47.0,H-SC,3.58,200857.0,-41320.0,152089.0,-2.87,-17.06,75.72,45.74,158.0,-0.27,1.40,30.64,XY24,NTT,ENGINEERING
53,QUESS,986.00,-49.31,36.0,X-SC,37.76,50304.0,-14702.0,165631.0,-2.43,-22.62,329.26,232.18,198.0,-0.09,0.35,0.00,XY24,NTT,MISC
16,CAMS,4762.00,-6.27,45.0,X-SC,0.79,250998.0,-858.0,63302.0,-2.42,-0.34,25.22,24.79,122.0,-0.01,1.75,22.26,X40N,NTT,MISC
49,MASFIN,398.61,-17.34,55.0,H-SC,8.79,93810.0,-4170.0,25770.0,-2.34,-4.26,27.47,22.05,152.0,-0.16,0.65,36.36,XR,ATH,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RECLTD,446.00,45.36,48.0,M-LC,4.96,204022.0,852.0,41274.0,-2.18,0.42,20.23,20.74,55.0,0.02,1.42,7.31,XY25,NTT,FINANCE
31,HINDZINC,730.22,26.60,46.0,M-LC,10.68,205929.0,853.0,111716.0,-1.51,0.42,54.25,54.89,52.0,0.01,1.44,24.61,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,103.63,54.0,M-SC,1.72,87770.0,-13007.0,13104.0,-1.28,-12.91,14.93,0.09,245.0,-0.99,0.61,15.31,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.42,49.0,H-SC,2.69,224100.0,-45567.0,81909.0,-0.06,-16.90,36.55,13.47,138.0,-0.56,1.56,13.95,XY24,NTT,PAINTS
17,CERA,9475.0,-20.53,61.0,H-SC,1.91,144877.0,-31026.0,73047.0,0.25,-17.64,50.42,23.89,149.0,-0.42,1.01,25.12,OX40N,NTT,CERAMICS
73,TATAELXSI,9161.0,-24.55,43.0,H-MC,6.04,102125.0,-29710.0,71937.0,-1.09,-22.54,70.44,32.03,98.0,-0.41,0.71,14.39,OX40N,NTT,IT
42,JCHAC,2282.0,19921.74,62.0,M-SC,1.50,97832.0,-29973.0,29956.0,1.23,-23.45,30.62,-0.01,232.0,-1.00,0.68,10.78,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-5.43,67.0,M-LC,10.18,172256.0,31416.0,13074.0,0.41,22.31,7.59,31.59,66.0,2.4,1.2,54.7,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,50.76,61.0,M-MC,9.76,237980.0,13018.0,154639.0,0.17,5.79,64.98,74.53,192.0,0.08,1.66,39.91,XY24,BTT,STEEL
56,RECLTD,446.00,45.36,48.0,M-LC,4.96,204022.0,852.0,41274.0,-2.18,0.42,20.23,20.74,55.0,0.02,1.42,7.31,XY25,NTT,FINANCE
31,HINDZINC,730.22,26.60,46.0,M-LC,10.68,205929.0,853.0,111716.0,-1.51,0.42,54.25,54.89,52.0,0.01,1.44,24.61,X5K,ATH,METALS
46,KPIGREEN,731.05,11.71,68.0,H-SC,6.47,133308.0,8011.0,50910.0,-1.82,6.39,38.19,47.03,141.0,0.16,0.93,66.91,MH,ATH,POWER
79,UNIONBANK,163.00,-5.43,67.0,M-LC,10.18,172256.0,31416.0,13074.0,0.41,22.31,7.59,31.59,66.0,2.40,1.20,54.70,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,11.71,68.0,H-SC,6.47,133308.0,8011.0,50910.0,-1.82,6.39,38.19,47.03,141.0,0.16,0.93,66.91,MH,ATH,POWER
85,WHIRLPOOL,2270.00,-39.02,58.0,M-SC,4.64,126500.0,11702.0,82339.0,-0.24,10.19,65.09,81.92,223.0,0.14,0.88,49.76,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.33,54.0,H-SC,6.07,130120.0,6784.0,124837.0,-0.47,5.50,95.94,106.72,119.0,0.05,0.91,30.16,AR,ATH,MISC
31,HINDZINC,730.22,26.60,46.0,M-LC,10.68,205929.0,853.0,111716.0,-1.51,0.42,54.25,54.89,52.0,0.01,1.44,24.61,X5K,ATH,METALS
86,WIPRO,420.00,-16.06,39.0,M-LC,5.40,147746.0,-3199.0,112656.0,-0.91,-2.12,76.25,72.51,53.0,-0.03,1.03,3.78,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-51.85,23.0,X-LC,26.39,151371.0,-125152.0,235231.0,1.71,-45.26,155.40,39.81,54.0,-0.53,1.06,6.69,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-40.87,27.0,X-SC,6.34,88395.0,-40275.0,85575.0,-0.51,-31.30,96.81,35.20,219.0,-0.47,0.62,1.46,X40,NTT,FOOTWEAR
44,JSWINFRA,342.00,-22.12,32.0,X-MC,4.84,195581.0,-4766.0,35263.0,-0.07,-2.38,18.03,15.23,178.0,-0.14,1.36,29.10,X40N,NTT,REALTY
30,HINDUNILVR,2922.00,-12.96,34.0,X-LC,7.02,244390.0,-12673.0,47803.0,-0.65,-4.93,19.56,13.67,24.0,-0.27,1.70,14.47,XY25,NTT,FMCG
51,PAGEIND,51769.92,-29.12,35.0,X-MC,7.10,80140.0,-3370.0,23401.0,-1.69,-4.04,29.20,23.98,82.0,-0.14,0.56,1.55,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,ITC,452.00,-37.92,47.0,X-LC,1.17,199903.0,-235.0,21130.0,-1.24,-0.12,10.57,10.44,4.0,-0.01,1.39,5.44,X40,NTT,FMCG
66,SIEMENS,4671.50,-2.83,47.0,H-LC,1.24,156230.0,-29865.0,77349.0,-0.09,-16.05,49.51,25.51,15.0,-0.39,1.09,15.13,AR,ATH,ELECTRICAL
52,PGHH,17907.65,-30.68,38.0,X-MC,1.28,200700.0,-120.0,67917.0,-0.08,-0.06,33.84,33.76,80.0,-0.00,1.40,5.13,X40,ATH,FMCG
28,HAVELLS,2069.16,-6.60,44.0,X-MC,2.92,311430.0,-16561.0,123108.0,-0.96,-5.05,39.53,32.48,92.0,-0.13,2.17,5.80,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-4.81,74.0,X-LC,4.25,249555.0,15411.0,33216.0,0.23,6.58,13.31,20.77,86.0,0.46,1.74,13.96,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-49.31,36.0,X-SC,37.76,50304.0,-14702.0,165631.0,-2.43,-22.62,329.26,232.18,198.0,-0.09,0.35,0.00,XY24,NTT,MISC
12,BATAINDIA,2096.00,-40.87,27.0,X-SC,6.34,88395.0,-40275.0,85575.0,-0.51,-31.30,96.81,35.20,219.0,-0.47,0.62,1.46,X40,NTT,FOOTWEAR
51,PAGEIND,51769.92,-29.12,35.0,X-MC,7.10,80140.0,-3370.0,23401.0,-1.69,-4.04,29.20,23.98,82.0,-0.14,0.56,1.55,X40,ATH,APPARELS
3,ACC,3906.00,-38.50,43.0,X-MC,2.46,182900.0,-55051.0,207701.0,-2.22,-23.14,113.56,64.15,174.0,-0.27,1.28,2.60,XY24,BTT,CEMENT
20,COLPAL,3726.84,-6.08,43.0,X-MC,7.35,219100.0,-44265.0,153589.0,-0.43,-16.81,70.10,41.51,84.0,-0.29,1.53,2.82,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-49.31,36.0,X-SC,37.76,50304.0,-14702.0,165631.0,-2.43,-22.62,329.26,232.18,198.0,-0.09,0.35,0.00,XY24,NTT,MISC
51,PAGEIND,51769.92,-29.12,35.0,X-MC,7.10,80140.0,-3370.0,23401.0,-1.69,-4.04,29.20,23.98,82.0,-0.14,0.56,1.55,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.17,42.0,X-SC,4.45,81592.0,-63568.0,139498.0,-0.10,-43.79,170.97,52.31,136.0,-0.46,0.57,9.22,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.86,39.0,X-MC,2.51,88095.0,-2583.0,17487.0,1.14,-2.85,19.85,16.44,101.0,-0.15,0.61,16.75,X40,ATH,PHARMA
12,BATAINDIA,2096.00,-40.87,27.0,X-SC,6.34,88395.0,-40275.0,85575.0,-0.51,-31.30,96.81,35.20,219.0,-0.47,0.62,1.46,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-29.65,45.0,X-LC,10.09,280844.0,-65112.0,131800.0,-0.96,-18.82,46.93,19.28,1.0,-0.49,1.96,3.82,X40,ATH,IT
39,INFY,2275.00,-19.60,48.0,X-LC,5.25,312684.0,-204.0,171882.0,-1.18,-0.07,54.97,54.87,3.0,-0.00,2.18,8.21,X40,BTT,IT
41,ITC,452.00,-37.92,47.0,X-LC,1.17,199903.0,-235.0,21130.0,-1.24,-0.12,10.57,10.44,4.0,-0.01,1.39,5.44,X40,NTT,FMCG
83,VBL,671.64,-16.58,54.0,X-LC,7.20,300498.0,-15344.0,128012.0,-0.79,-4.86,42.60,35.67,5.0,-0.12,2.10,8.32,X40N,ATH,FMCG
1,ABB,7934.00,-39.77,53.0,H-LC,5.22,251856.0,-9763.0,128975.0,-0.17,-3.73,51.21,45.57,7.0,-0.08,1.76,7.17,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-47.69,41.0,L-SC,28.06,77760.0,-35789.0,75839.0,-1.84,-31.52,97.53,35.27,268.0,-0.47,0.54,90.32,XR,NTT,HOTELS
7,ASIANTILES,137.00,7494.44,71.0,L-SC,9.85,83348.0,-10462.0,87082.0,1.42,-11.15,104.48,81.67,269.0,-0.12,0.58,61.41,XR,NTT,CERAMICS
49,MASFIN,398.61,-17.34,55.0,H-SC,8.79,93810.0,-4170.0,25770.0,-2.34,-4.26,27.47,22.05,152.0,-0.16,0.65,36.36,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-39.02,58.0,M-SC,4.64,126500.0,11702.0,82339.0,-0.24,10.19,65.09,81.92,223.0,0.14,0.88,49.76,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.33,54.0,H-SC,6.07,130120.0,6784.0,124837.0,-0.47,5.50,95.94,106.72,119.0,0.05,0.91,30.16,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VAIBHAVGBL,521.00,79.06,70.0,H-SC,1.72,155100.0,-27675.0,138737.0,-2.22,-15.14,89.45,60.77,125.0,-0.20,1.08,38.92,XR,NTT,JEWELLERY
59,REPCOHOME,880.00,-51.56,76.0,H-SC,6.76,276679.0,-9826.0,267161.0,2.18,-3.43,96.56,89.82,134.0,-0.04,1.93,46.38,XY24,NTT,FINANCE
46,KPIGREEN,731.05,11.71,68.0,H-SC,6.47,133308.0,8011.0,50910.0,-1.82,6.39,38.19,47.03,141.0,0.16,0.93,66.91,MH,ATH,POWER
68,SONACOMS,806.63,-32.06,66.0,M-SC,7.25,84621.0,-16538.0,56535.0,0.48,-16.35,66.81,39.54,202.0,-0.29,0.59,19.32,AR,ATH,AUTO
62,SAMMAANCAP,326.00,-196.59,70.0,M-SC,32.21,167778.0,18558.0,125615.0,-1.85,12.44,74.87,96.62,208.0,0.15,1.17,81.57,XY25,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.68
1,25,44.15
2,50,75.48


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.02
LC    32.96
MC    23.01
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.08
X40      16.10
X40N     12.44
XY25     11.89
XR        9.90
AR        7.84
OX40N     6.55
X200      1.77
X5K       1.44
SR        1.05
MH        0.93
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.39
X-LC    22.27
X-MC    18.69
M-SC    12.68
X-SC     5.50
M-LC     5.09
H-LC     4.62
H-MC     2.35
M-MC     1.66
L-SC     1.45
L-LC     0.98
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.24,-3.59,37.08
IT,12.36,-18.58,82.10
FINANCE,12.18,-8.13,57.14
MISC,7.74,-17.23,76.36
ELECTRICAL,5.79,-8.88,47.49
PAINTS,5.46,-17.76,35.32
BANKS,4.05,-16.19,83.78
INSURANCE,3.86,0.01,37.27
AUTO,2.78,-44.59,104.93


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3262972.0,22
XR,1234201.0,13
AR,1164490.0,9
X40,847187.0,12
X40N,684100.0,9
OX40N,544787.0,9
XY25,469570.0,8
SR,270776.0,2
X5K,111716.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3334087.0,24
M-SC,1226965.0,15
X-MC,1193097.0,14
X-LC,1085912.0,13
X-SC,560576.0,6
M-LC,278720.0,4
H-LC,277275.0,3
L-SC,252195.0,3
H-MC,250933.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1165952.0      6
           AR         834833.0      5
           XR         770657.0      7
M-SC       XY24       705170.0      6
X-MC       XY24       571274.0      4
X-LC       X40        472813.0      5
X-SC       X40N       309370.0      4
X-LC       XY24       308236.0      2
X-MC       X40        288799.0      6
H-SC       SR         270776.0      2
           OX40N      240959.0      3
X-LC       X40N       221555.0      3
H-LC       AR         206324.0      2
X-MC       XY25       179849.0      2
H-MC       XY24       178996.0      1
X-SC       XY24       165631.0      1
L-SC       XR         162921.0      2
M-MC       XY24       154639.0      1
X-MC       X40N       153175.0      2
M-SC       OX40N      142617.0      4
           XR         130230.0      2
           XY25       125615.0      1
           AR         123333.0      2
M-LC       XR         112656.0      1
           X5K        111716.0      1
L-SC       OX40N       89274.0      1
X-SC       X40         85575.0      1
X-LC       XY25        83308.0      3
H-MC       OX40N       71937.0      1
H-LC       X200        70951.0      1
L-MC       XR          57737.0      1
H-SC       MH          50910.0      1
M-LC       XY25        41274.0      1
L-LC       XY25        39524.0      1
M-LC       XY24        13074.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 42.0 seconds
